In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size : list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [6]:
from braintumorClassifier.constants import *
from braintumorClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

[2024-02-10 16:03:37,140: INFO: __init__: Starting data processing]
[2024-02-10 16:03:37,142: INFO: __init__: Data preprocessing completed successfully.]
[2024-02-10 16:03:42,547: WARNING: module_wrapper: From c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-02-10 16:03:42,714: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAM_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'data')
        create_directories([
            Path(training.root_dir)
        ])
        print("Before TrainingConfig initialization")


        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
        )
        return training_config
        print("After TrainingConfig initialization")

    

In [8]:
import time 
import os
import urllib.request as request
from zipfile import ZipFile
# import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [9]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}',
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = 'categorical'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

       

# Pass the learning rate scheduler to the list of callbacks
        # callback_list.append(lr_scheduler)

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40, 
                horizontal_flip = True, 
                width_shift_range = 0.2, 
                height_shift_range = 0.2, 
                shear_range = 0.2, 
                zoom_range = 0.2, 
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        self.train_generator = train_datagenerator.flow_from_directory(directory = self.config.training_data,
                                                                            subset = 'training', shuffle = True,
                                                                            **dataflow_kwargs )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):

        callback_list = [
            EarlyStopping(
                monitor = 'val_loss', patience=10, verbose=1, restore_best_weights=True), 
                ModelCheckpoint(
                    filepath=str(self.config.trained_model_path),
                    monitor = 'val_loss', save_best_only=True, verbose=1),
                    ReduceLROnPlateau(
                        monitor = 'val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)
        ]
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator, 
            epochs = self.config.params_epochs, 
            steps_per_epoch = self.step_per_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [15]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list)
except Exception as e:
    raise e


[2024-02-10 16:04:22,151: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 16:04:22,154: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 16:04:22,155: INFO: common: created directory at: artifacts]
[2024-02-10 16:04:22,158: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-10 16:04:22,160: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-02-10 16:04:22,163: INFO: common: created directory at: artifacts\training]
Before TrainingConfig initialization
Found 208 images belonging to 2 classes.
Found 833 images belonging to 2 classes.
Epoch 1/10
[2024-02-10 16:04:26,639: WARNING: module_wrapper: From c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
52/52 [==============================] - ETA: 0s - loss: 84.5029 -

c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52/52 [==============================] - 84s 2s/step - loss: 84.5029 - accuracy: 0.6683 - val_loss: 22.0765 - val_accuracy: 0.7933 - lr: 0.0100
Epoch 2/10
52/52 [==============================] - ETA: 0s - loss: 33.0067 - accuracy: 0.7760
Epoch 2: val_loss did not improve from 22.07654
52/52 [==============================] - 80s 2s/step - loss: 33.0067 - accuracy: 0.7760 - val_loss: 27.8162 - val_accuracy: 0.7596 - lr: 0.0100
Epoch 3/10
52/52 [==============================] - ETA: 0s - loss: 40.5772 - accuracy: 0.7699
Epoch 3: val_loss did not improve from 22.07654
52/52 [==============================] - 80s 2s/step - loss: 40.5772 - accuracy: 0.7699 - val_loss: 24.2353 - val_accuracy: 0.8654 - lr: 0.0100
Epoch 4/10
52/52 [==============================] - ETA: 0s - loss: 48.2008 - accuracy: 0.7772
Epoch 4: val_loss improved from 22.07654 to 21.86102, saving model to artifacts\training\model.h5
52/52 [==============================] - 77s 1s/step - loss: 48.2008 - accuracy: 0.7772 -